# Eigenmode Simulation of Two Capacitively Coupled Transmons

### Qiskit Metal Design

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

design = designs.DesignPlanar({}, True)

#Set up chip dimensions 
design.chips.main.size.size_x = '2.2mm'
design.chips.main.size.size_y = '1.8mm'
design.chips.main.size.size_z = '-500um'
design.chips.main.size.center_x = '0.5mm'
design.chips.main.size.center_y = '0mm'

#Create gui to visualise design
gui = MetalGUI(design)

#Transmon 1
q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um',
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

#Transmon 2
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x = '1.0 mm',
    pad_width = '425 um',
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=-1,loc_H=+1, pad_width='200um')
    )))

#Coupler for transmons
coupler = RouteStraight(design, 'coupler', Dict(hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='Q2', pin='readout')), ))

#Rebuild circuit
gui.rebuild()
gui.autoscale()

### Eigenmode Simulation
Remember to use your path for the palace binary under "palace_dir"

In [ ]:
from SQDMetal.PALACE.Eigenmode_Simulation import PALACE_Eigenmode_Simulation
from SQDMetal.Utilities.Materials import MaterialInterface

#Eigenmode Simulation Options
user_defined_options = {
                "mesh_refinement":  0,                             #refines mesh in PALACE - essetially divides every mesh element in half
                "dielectric_material": "silicon",                  #choose dielectric material - 'silicon' or 'sapphire'
                "starting_freq": 5.0e9,                            #starting frequency in Hz 
                "number_of_freqs": 2,                              #number of eigenmodes to find
                "solns_to_save": 2,                                #number of electromagnetic field visualizations to save
                "solver_order": 2,                                 #increasing solver order increases accuracy of simulation, but significantly increases sim time
                "solver_tol": 1.0e-10,                             #error residual tolerance foriterative solver
                "solver_maxits": 250,                              #number of solver iterations
                "fillet_resolution":12,                            #number of vertices per quarter turn on a filleted path
                "palace_dir":"~/spack/opt/spack/linux-ubuntu24.04-zen2/gcc-13.3.0/palace-develop-36rxmgzatchgymg5tcbfz3qrmkf4jnmj/bin/palace", #"PATH/TO/PALACE/BINARY",
                "num_cpus": 16
                }

#Create the Palace Eigenmode simulation
eigen_sim = PALACE_Eigenmode_Simulation(name ='two_transmon_test',                                      #name of simulation
                                        metal_design = design,                                          #feed in qiskit metal design
                                        sim_parent_directory = "",                                      #choose directory where mesh file, config file and HPC batch file will be saved
                                        mode = 'simPC',                                                 #choose simulation mode 'HPC' or 'simPC'                                          
                                        meshing = 'GMSH',                                               #choose meshing 'GMSH' or 'COMSOL'
                                        user_options = user_defined_options,                            #provide options chosen above
                                        create_files = True)                                            #create mesh, config and HPC batch files
#add in metals from the first layer
eigen_sim.add_metallic(1)

#add ground plane into simulation
eigen_sim.add_ground_plane()

#Create lumped element ports for each JJ
eigen_sim.create_port_JosephsonJunction('Q1', L_J=13e-9, C_J = 0e-15)
eigen_sim.create_port_JosephsonJunction('Q2', L_J=9e-9, C_J = 0e-15)

#Fine mesh the qubits (only qubit metal regions i.e. metals_only = True) and coupler
eigen_sim.fine_mesh_around_comp_boundaries(['Q1', 'Q2', 'coupler'], min_size=8e-6, max_size=150e-6, taper_dist_min=12e-6, metals_only=True)

#Run post-processing for participation ratios for lossy interfaces
eigen_sim.setup_EPR_interfaces(metal_air=MaterialInterface('Aluminium-Vacuum'), substrate_air=MaterialInterface('Silicon-Vacuum'), substrate_metal=MaterialInterface('Silicon-Aluminium'))

#Prepares the mesh file and config file
eigen_sim.prepare_simulation()

In [3]:
#Inspect the mesh before running the simulation
eigen_sim.open_mesh_gmsh()

In [ ]:
#run the simulation
eigen_sim.run()